In [25]:
import numpy as np
from scipy.stats import norm


def portfolio_projection(
    C, N, mu, sigma, percentiles=[10, 50, 90], *, L=0.0, L_year_end=1
):
    """
    Semi-analytical percentiles for terminal value after N years with:
      - Annual contribution C at the end of each year (ordinary annuity)
      - One-time lump sum L contributed at the END of year `L_year_end`
        (i.e., after that year's return is realized). The lump then
        compounds for (N - L_year_end) years. If L_year_end > N, L is not
        deposited within the horizon.

    Parameters
    ----------
    C : float
        Annual contribution (end of each year 1..N).
    N : int
        Horizon in years.
    mu : float
        Expected *arithmetic* simple return per year (e.g., 0.06).
    sigma : float
        Volatility of *arithmetic* simple returns per year (e.g., 0.15).
    percentiles : list[float]
        Percentiles in [0, 100] to return (e.g., [5, 50, 95]).
    L : float, optional
        One-time lump sum amount (default 0.0).
    L_year_end : int, optional
        Year index (1..N) when the lump is deposited at year-end.
        Default 1 (end of year 1).

    Returns
    -------
    dict[float, float]
        Mapping percentile -> terminal projected value.

    Notes
    -----
    - Uses moment-matching to lognormal for each component separately.
    - Percentiles for the total use comonotonic coupling (ρ=1): same Z
      for both components, giving valid quantiles for the sum.
    - p=50 returns the *median* (not the mean). If you also need means,
      compute Ea+El from the helpers below.
    """

    # ----- helpers -----
    def annuity_moments(C, N, mu, sigma):
        if N <= 0 or C == 0.0:
            return 0.0, 0.0
        m = 1.0 + mu
        A = m**2 + sigma**2

        # mean
        if abs(mu) < 1e-14:
            E = C * N
        else:
            E = C * (m**N - 1.0) / mu

        # second moment pieces
        if abs(A - 1.0) < 1e-14 and abs(mu) < 1e-14 and abs(sigma) < 1e-14:
            # deterministic case
            S = float(N)
            T = N * (N - 1) / 2.0
        else:
            S = (A**N - 1.0) / (A - 1.0)
            T = 0.0
            for p in range(1, N):
                T += (m**p) * ((A ** (N - p) - 1.0) / (A - 1.0))

        second_moment = C**2 * (S + 2.0 * T)
        V = max(0.0, second_moment - E**2)
        return E, V

    def lump_moments_end_of_year(L, N, mu, sigma, year_end):
        # If lump happens after the horizon, it contributes nothing yet
        if L == 0.0 or year_end is None or year_end > N:
            return 0.0, 0.0
        # Years the lump compounds after being deposited at end of `year_end`
        nL = max(0, N - int(year_end))
        if nL == 0:
            # Deposited at the end of year N -> no compounding
            return float(L), 0.0

        m = 1.0 + mu
        A = m**2 + sigma**2
        E = L * (m**nL)
        second_moment = (L**2) * (A**nL)
        V = max(0.0, second_moment - E**2)
        return E, V

    def moments_to_lognormal(E, V):
        if E <= 0.0:
            return -np.inf, 0.0  # exp(-inf) -> 0
        if V <= 0.0:
            return np.log(E), 0.0
        s2 = np.log(1.0 + V / (E**2))
        return (np.log(E) - 0.5 * s2, np.sqrt(s2))

    # ----- component moments -----
    Ea, Va = annuity_moments(C, N, mu, sigma)
    El, Vl = lump_moments_end_of_year(L, N, mu, sigma, L_year_end)

    # lognormal params for each component
    mu_a, sg_a = moments_to_lognormal(Ea, Va)
    mu_l, sg_l = moments_to_lognormal(El, Vl)

    # ----- comonotonic (ρ=1) percentiles for the sum -----
    out = {}
    for p in percentiles:
        # guard against p=0 or 100
        q = min(max(p, 1e-12), 100 - 1e-12) / 100.0
        z = norm.ppf(q)
        Qa = 0.0 if np.isneginf(mu_a) else np.exp(mu_a + z * sg_a)
        Ql = 0.0 if np.isneginf(mu_l) else np.exp(mu_l + z * sg_l)
        out[p] = float(Qa + Ql)

    return out

In [76]:
def mc_portfolio_projection(
    L,
    C,
    N,
    mu,
    sigma,
    num_sims=100_000,
    seed=None,
    *,
    model="lognormal",  # "lognormal" or "normal"
    L_year_end=1,  # None or <=0 => deposit at t=0 (original behavior).
    # If 1..N => deposit at END of that year; if >N => no deposit within horizon.
):
    """
    Monte Carlo terminal FV for:
    - Lump sum L (at t=0 by default, or at END of year L_year_end if specified)
    - End-of-year contribution C for N years

    Parameters
    ----------
    L : float
        One-time lump sum amount.
    C : float
        Annual contribution (end of each year 1..N).
    N : int
        Horizon in years.
    mu : float
        Expected return per step. If model="lognormal", interpret as mean of log gross.
        If model="normal", interpret as arithmetic simple-return mean.
    sigma : float
        Volatility per step. If model="lognormal", stdev of log gross; otherwise simple-return stdev.
    num_sims : int
        Number of simulation paths.
    seed : int or None
        RNG seed for reproducibility.
    model : {"lognormal","normal"}
        Return model.
    L_year_end : int or None
        - None or <=0: deposit at t=0 (before any returns).
        - 1..N: deposit at the END of that year (after that year's return).
        - >N: deposit occurs after the horizon -> contributes 0 within horizon.

    Returns
    -------
    dict
        {"mean": ..., 50: median, 10: p10, 90: p90}
    """
    rng = np.random.default_rng(seed)

    if N <= 0:
        # No return periods: only whatever has already been deposited matters
        # If L_year_end is None/<=0, L is already in; if L_year_end >= 1, it hasn't happened yet.
        L_now = float(L) if (L_year_end is None or L_year_end <= 0) else 0.0
        portfolio = np.full(num_sims, L_now, dtype=float)
        return {
            "mean": float(portfolio.mean()),
            50: float(np.percentile(portfolio, 50)),
            10: float(np.percentile(portfolio, 10)),
            90: float(np.percentile(portfolio, 90)),
        }

    # Draw gross returns
    if model == "lognormal":
        # Interpret mu, sigma as parameters for log gross
        Z = rng.standard_normal(size=(num_sims, N))
        growth = np.exp((mu - 0.5 * sigma**2) + sigma * Z)  # strictly positive
    else:
        # Simple-return normal
        R = rng.normal(mu, sigma, size=(num_sims, N))
        growth = 1.0 + R

    # Cumulative gross to end-of-year k (columns: year 1..N)
    cum = np.cumprod(growth, axis=1)  # shape (num_sims, N)

    # ----- Lump sum FV with flexible timing -----
    if L == 0.0:
        fv_lump = np.zeros(num_sims, dtype=float)
    else:
        if L_year_end is None or L_year_end <= 0:
            # Deposit at t=0: compounds for all N years
            fv_lump = L * cum[:, -1]
        elif L_year_end > N:
            # Deposit happens after the horizon: contributes nothing yet
            fv_lump = np.zeros(num_sims, dtype=float)
        else:
            # Deposit at END of year y (1..N) -> compounds for years y+1..N
            # Factor = product_{t=y+1..N} growth_t = cum[:, -1] / cum[:, y-1]
            y_idx = int(L_year_end) - 1  # 0-based index for the year just ended
            factor = cum[:, -1] / cum[:, y_idx]
            fv_lump = L * factor

    # ----- Ordinary annuity FV at year N -----
    # sum_{j=1..N} C * prod_{t=j..N} growth_t
    tail_prod = np.cumprod(growth[:, ::-1], axis=1)[:, ::-1]  # shape (num_sims, N)
    fv_ann = C * tail_prod.sum(axis=1)

    portfolio = fv_lump + fv_ann

    return {
        "mean": float(portfolio.mean()),
        50: float(np.percentile(portfolio, 50)),
        10: float(np.percentile(portfolio, 10)),
        90: float(np.percentile(portfolio, 90)),
    }

In [80]:
C = 200_000
# C = 0
# L = 0
L = 1_000_000
N = 25
mu = 0.06
sigma = 0.15
percentiles = [10, 50, 90]
num_sims = 200_000
seed = 42

In [81]:
res = mc_portfolio_projection(
    L=L,
    C=C,
    N=N,
    mu=mu,
    sigma=sigma,
    num_sims=num_sims,
    seed=seed,
    L_year_end=1,
    model="normal",
)

print(f"Median: {res[50]:,.0f}")
print(f"pct10: {res[10]:,.0f}")
print(f"pct90: {res[90]:,.0f}")

Median: 13,538,695
pct10: 6,926,605
pct90: 27,143,737


In [82]:
res = portfolio_projection(
    L=L, C=C, N=N, mu=mu, sigma=sigma, percentiles=percentiles, L_year_end=1
)

print(f"Median: {res[50]:,.0f}")
print(f"pct10: {res[10]:,.0f}")
print(f"pct90: {res[90]:,.0f}")

Median: 12,987,570
pct10: 6,638,572
pct90: 25,763,262
